In [15]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [16]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


In [17]:
input = Variable(torch.randn(1, 1, 32, 32))
out = net(input)
print(out)

Variable containing:
-0.0242  0.1074  0.0545  0.0107  0.0840  0.0010 -0.0112  0.0178 -0.0926 -0.0614
[torch.FloatTensor of size 1x10]



In [18]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [19]:
output = net(input)
target = Variable(torch.arange(1, 11))
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

Variable containing:
 38.6155
[torch.FloatTensor of size 1]



In [20]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

In [21]:
net.zero_grad() #!!! Clear the gradient before applying backprop. If not cleaned grad will accumulate

print('conv1.bias.grad before backprop')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backprop')
print(net.conv1.bias.grad)


conv1.bias.grad before backprop
Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

conv1.bias.grad after backprop
Variable containing:
1.00000e-02 *
  9.4958
  2.8798
  4.5870
  3.2107
  1.6578
  5.0080
[torch.FloatTensor of size 6]



# Updating the weights

1. Simple update rule:
    ´´´
    weight = weight - learning_rate * gradient
    ´´´

In [22]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

2. Update with complex algorithm:
 - SGD
 - Adam
 - momentum

In [23]:
import torch.optim as optim

#create your optimiser
optimizer = optim.SGD(net.parameters(), lr = 0.01)

#in your training loop
optimizer.zero_grad()       # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()


# Conclusion 
1. Very important is that gradient is zeroed out before each backprop step, because gradients are accumulative.